In [72]:
import os, sys
import numpy as np
from tools import *
from image_processing import *
from optparse import OptionParser

from model import *

from keras import backend


In [73]:
limit = 11500
backend.set_image_dim_ordering('th')
IMG_SIZE = 224

In [163]:
if os.path.exists('preprocessed_cats.pckl') and True:

    print ("Loading preprocessed images....")
    preprocessed_cats = restore('preprocessed_cats.pckl')
    preprocessed_dogs = restore('preprocessed_dogs.pckl')
    limit = len(preprocessed_cats)
    
    print ("Loading done.")
else:
    print ("Preprocessing....")
    # cats : 0
    print ("Preprocessing cats images:")
    preprocessed_cats = preprocess_imgs('train/cats', IMG_SIZE, limit = limit)
    store(preprocessed_cats, 'preprocessed_cats.pckl')

    # dogs : 1
    print ("Preprocessing dogs images:")
    preprocessed_dogs = preprocess_imgs('train/dogs', IMG_SIZE, limit = limit)
    store(preprocessed_dogs, 'preprocessed_dogs.pckl')
    limit = len(preprocessed_cats)
    

Preprocessing....
Preprocessing cats images:
[============================================================] 100.0% ...

Preprocessing dogs images:
[============================================================] 100.0% ...



In [164]:
# labelling the data
# cats = [1 0] and dogs = [0 1]
zeros = np.zeros((limit, 1))
ones  = np.ones((limit, 1))
Ycats = np.array((ones, zeros))
print (Ycats.shape)
Ydogs = np.array((zeros, ones))
print (Ydogs.shape)

print ("Ycats", Ycats[:,0])
print ("Ydogs", Ydogs[:,0])

Y = np.concatenate((Ycats, Ydogs), axis=1)
Y = np.swapaxes(Y, 0, 1).squeeze()
print (Y.shape)


print ("Y = ", Y[0,:])

(2, 11500, 1)
(2, 11500, 1)
('Ycats', array([[ 1.],
       [ 0.]]))
('Ydogs', array([[ 0.],
       [ 1.]]))
(23000, 2)
('Y = ', array([ 1.,  0.]))


In [165]:


X = np.concatenate((preprocessed_cats, preprocessed_dogs))

print X.shape

X = np.swapaxes(X, 1, 3)

print X.shape
print("(23000, 3, 224, 224)")

(23000, 224, 224, 3)
(23000, 3, 224, 224)
(23000, 3, 224, 224)


# Construct and test Vgg model

In [206]:
model = create_model(IMG_SIZE)
for layer in model.layers: layer.trainable=False

In [193]:
classes = get_classes()


def predict_cat_dog_at_index(model, input_array, index, limit = limit):
    print ("Predict a cat: ")
    cat_prediction = model.predict(np.array([input_array[index,:,:]])).argmax(axis = 1)[0]
    print ( cat_prediction, classes[cat_prediction] )

    print ("Predict a dog: ")
    dog_prediction = model.predict(np.array([input_array[limit + index,:,:]])).argmax(axis = 1)[0]
    print (dog_prediction, classes[dog_prediction])
    
predict_cat_dog_at_index(model, X, 569)

Predict a cat: 
(233, u'Bouvier_des_Flandres')
Predict a dog: 
(265, u'toy_poodle')


# Sepate convolutional layers and dense layers


In [214]:
print ("Total number of layers : {}, only the 6 last layers are dense layers".format(len(model.layers)))

def no_process(x):
    # return x[:, ::-1]
    return x

nb_layers = len(model.layers)

dense_model = Sequential()
dense_model.add(Lambda(no_process, input_shape=(512, 7, 7), output_shape=(512, 7, 7) ))
for layer in model.layers[-6:]:
    if "dropout" not in layer.name:
        dense_model.add(layer)
convolutional_model = Sequential(model.layers[:nb_layers - 6])

# convolutional_model = Sequential()
# dense_model = Sequential()
# dense_model.add( Lambda(no_process, input_shape=(512, 7, 7), output_shape=(512, 7, 7) ) )


#for i,layer in enumerate(model.layers):
#    if i < len(model.layers) - 6:
#        convolutional_model.add(layer)
#    else:
#        dense_model.add(layer)

print ("Total number of convolutional layers : {}".format(len(convolutional_model.layers)))
print ("Total number of dense layers : {}".format(len(dense_model.layers)))

Total number of layers : 38, only the 6 last layers are dense layers
Total number of convolutional layers : 32
Total number of dense layers : 5


In [186]:
#convolutional_model.summary()

In [215]:
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_9 (Lambda)            (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 4096)              102764544 
_________________________________________________________________
dense_18 (Dense)             (None, 4096)              16781312  
_________________________________________________________________
dense_19 (Dense)             (None, 1000)              4097000   
Total params: 123,642,856
Trainable params: 0
Non-trainable params: 123,642,856
_________________________________________________________________


In [188]:
# perdict and save the output of the convolutional layers

if os.path.exists('conv_output.pckl') :
    conv_output = restore('conv_output.pckl')
else:
    conv_output = convolutional_model.predict(X)
    store(conv_output, 'conv_output.pckl')
print (conv_output.shape)

#resore the output if don't want to recompute the cell above


(23000, 512, 7, 7)


In [216]:
# check that the output still makes sense
predict_cat_dog_at_index(dense_model, conv_output, 13)

Predict a cat: 
(811, u'space_heater')
Predict a dog: 
(195, u'Boston_bull')


In [196]:
im = Image.fromarray(np.swapaxes(X[12,:,:], 0 , 2) )
# im = Image.open("train/cats/cat.50.jpg")
im.show()

# Finetune the dense model to detect cats and dogs

In [ ]:
print (dense_model.layers[-1])
dense_model.pop()

for layer in dense_model.layers: layer.trainable=False

In [198]:
#dense_model.pop()
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_5 (Lambda)            (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 4096)              102764544 
_________________________________________________________________
dense_11 (Dense)             (None, 4096)              16781312  
Total params: 119,545,856
Trainable params: 0
Non-trainable params: 119,545,856
_________________________________________________________________


In [217]:
dense_model.add(Dense(2, activation='softmax', input_shape=(1000,)))
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_9 (Lambda)            (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 4096)              102764544 
_________________________________________________________________
dense_18 (Dense)             (None, 4096)              16781312  
_________________________________________________________________
dense_19 (Dense)             (None, 1000)              4097000   
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 2002      
Total params: 123,644,858
Trainable params: 2,002
Non-trainable params: 123,642,856
__________________________________________________________

In [222]:
dense_model.compile(
        # loss='binary_crossentropy',
       #optimizer=SGD(lr=10.0),
          #optimizer=RMSprop(),
       optimizer=Adam(lr=0.0001),
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [201]:
dense_model.load_weights("dense_model.h5")

print ("Y = ", Y[0:10,:])

print ("Y = ", Y[limit:limit+10,:])

('Y = ', array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]]))
('Y = ', array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]]))


In [223]:
dense_model.fit(conv_output,Y, epochs=5, batch_size=100)
dense_model.save_weights("dense_model.h5")
dense_model.layers[-1].get_weights()

Epoch 1/5
23000/23000 [==============================] - 6s - loss: 0.6851 - acc: 0.7233     
Epoch 2/5
23000/23000 [==============================] - 6s - loss: 0.6752 - acc: 0.8472     
Epoch 3/5
23000/23000 [==============================] - 6s - loss: 0.6655 - acc: 0.8714     
Epoch 4/5
23000/23000 [==============================] - 6s - loss: 0.6560 - acc: 0.8796     
Epoch 5/5
23000/23000 [==============================] - 6s - loss: 0.6468 - acc: 0.8817     


[array([[ 0.03199724,  0.0544106 ],
        [ 0.07699834, -0.05127247],
        [-0.1129072 ,  0.04124811],
        ..., 
        [-0.01257063,  0.05676379],
        [ 0.07193685, -0.15178901],
        [ 0.14304245, -0.13594384]], dtype=float32),
 array([-0.0032438,  0.0032438], dtype=float32)]

In [224]:
cat_dog_class = {0: 'cat', 1: 'dog'}

print ("Predict a cat: ")
cat_number = 50
cat_prediction = dense_model.predict(conv_output[cat_number,:,:].reshape(1,512, 7, 7))
cat_prediction_decision = cat_prediction.argmax(axis = 1)[0]
print ( cat_prediction_decision, cat_dog_class[cat_prediction_decision], cat_prediction )
print (cat_prediction)
print ("Y = ", Y[cat_number])

print ("Predict a dog: ")
dog_number = 1701
dog_prediction = dense_model.predict(conv_output[limit+dog_number,:,:].reshape(1,512, 7, 7))
dog_prediction_decision = dog_prediction.argmax(axis = 1)[0]
print (dog_prediction_decision, cat_dog_class[dog_prediction_decision], dog_prediction)
print (dog_prediction)
print ("Y = ", Y[limit+dog_number])

Predict a cat: 
(0, 'cat', array([[ 0.53970671,  0.46029332]], dtype=float32))
[[ 0.53970671  0.46029332]]
('Y = ', array([ 1.,  0.]))
Predict a dog: 
(1, 'dog', array([[ 0.48492044,  0.51507962]], dtype=float32))
[[ 0.48492044  0.51507962]]
('Y = ', array([ 0.,  1.]))


# Finetune the original model to detect cats and dogs

In [204]:
model = create_model(IMG_SIZE)
for layer in model.layers: layer.trainable=False

predict_cat_dog_at_index(model, X, 13)

Predict a cat: 
(811, u'space_heater')
Predict a dog: 
(195, u'Boston_bull')


In [209]:
#model.pop()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_7 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_66 (ZeroPaddi (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_67 (ZeroPaddi (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_68 (ZeroPaddi (None, 64, 114, 114)      0         
__________

In [29]:
model.add(Dense(2, activation='sigmoid', input_shape=(1000,)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_5 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_40 (ZeroPaddi (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_41 (ZeroPaddi (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_42 (ZeroPaddi (None, 64, 114, 114)      0         
__________

In [30]:
model.compile(
        # loss='binary_crossentropy',
          # optimizer=SGD(lr=10.0),
          #optimizer=RMSprop(),
         optimizer=Adam(lr=0.001),
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [31]:
model.load_weights("model.h5")

In [32]:
model.fit(X,Y, epochs=1, batch_size=50)
model.save_weights("model.h5")

Epoch 1/1
23000/23000 [==============================] - 415s - loss: 0.6933 - acc: 0.4925   


In [27]:
cat_dog_class = {0: 'cat', 1: 'dog'}

print ("Predict a cat: ")
print (model.predict(X[50,:,:].reshape(1,3,224,224)))
cat_prediction = model.predict(X[50,:,:].reshape(1,3,224,224)).argmax(axis = 1)[0]
print ( cat_prediction, cat_dog_class[cat_prediction] )

print ("Predict a dog: ")
dog_prediction = model.predict(X[1701,:,:].reshape(1,3,224,224)).argmax(axis = 1)[0]
print (dog_prediction, cat_dog_class[dog_prediction])

Predict a cat: 
[[  4.00468116e-06   1.03098873e-05   9.88933061e-06   7.62894342e-06
    6.57140026e-06   4.05430437e-05   5.65189112e-06   8.02692193e-06
    3.23290988e-05   5.09283473e-06   1.61529370e-05   1.94507793e-05
    1.30366185e-04   4.56952403e-05   3.93438677e-06   1.86773450e-05
    7.71370105e-05   1.37451334e-05   2.94764163e-06   2.85266015e-05
    2.20317106e-05   1.01240755e-06   1.03186665e-06   1.75560751e-06
    2.02863157e-06   2.81494567e-06   2.18191435e-05   7.17105650e-06
    3.18065645e-06   1.19487180e-04   1.71009415e-05   3.22048072e-05
    4.03311751e-05   1.08176810e-05   2.94680945e-06   5.36827683e-05
    2.40656838e-04   2.98556188e-05   3.85718704e-05   1.56009664e-05
    1.27088624e-05   8.13602219e-06   4.53189023e-06   2.49456607e-05
    7.14476555e-05   6.74303737e-06   5.60275694e-05   6.80388030e-06
    9.59397084e-07   7.59038244e-07   1.04444680e-05   2.93254416e-05
    1.59177125e-05   8.32210299e-06   2.53987382e-05   6.36722143e-06
    

KeyError: 435

# Create validation set

In [225]:
limit_valid = 1000
backend.set_image_dim_ordering('th')
IMG_SIZE = 224

if os.path.exists('preprocessed_validation_cats.pckl') and True:

    print ("Loading preprocessed images....")
    preprocessed_valid_cats = restore('preprocessed_validation_cats.pckl')
    preprocessed_valid_dogs = restore('preprocessed_validation_dogs.pckl')
    limit_valid = len(preprocessed_valid_cats)
    
    print ("Loading done.")
else:
    print ("Preprocessing....")
    # cats : 0
    print ("Preprocessing cats images:")
    preprocessed_valid_cats = preprocess_imgs('valid/cats', IMG_SIZE, limit = limit_valid)
    store(preprocessed_valid_cats, 'preprocessed_validation_cats.pckl')

    # dogs : 1
    print ("Preprocessing dogs images:")
    preprocessed_valid_dogs = preprocess_imgs('valid/dogs', IMG_SIZE, limit = limit_valid)
    store(preprocessed_valid_dogs, 'preprocessed_validation_dogs.pckl')
    limit_valid = len(preprocessed_valid_dogs)
    

Loading preprocessed images....
Loading done.


In [226]:
# labelling the data
# cats = [1 0] and dogs = [0 1]
zeros_valid = np.zeros((limit, 1))
ones_valid  = np.ones((limit, 1))
Ycats_valid = np.array((ones_valid, zeros_valid))
print (Ycats_valid.shape)
Ydogs_valid = np.array((zeros_valid, ones_valid))
print (Ydogs_valid.shape)

print ("Ycats", Ycats_valid[:,0])
print ("Ydogs", Ydogs_valid[:,0])

Y_valid = np.concatenate((Ycats_valid, Ydogs_valid), axis=1)
Y_valid = np.swapaxes(Y_valid, 0, 1).squeeze()
print (Y_valid.shape)


print ("Y = ", Y_valid[0,:])

(2, 11500, 1)
(2, 11500, 1)
('Ycats', array([[ 1.],
       [ 0.]]))
('Ydogs', array([[ 0.],
       [ 1.]]))
(23000, 2)
('Y = ', array([ 1.,  0.]))


In [227]:

Xcats_valid = np.array(preprocessed_valid_cats)
Xdogs_valid = np.array(preprocessed_valid_dogs)

X_valid = np.concatenate((Xcats_valid, Xdogs_valid))

X_valid = np.swapaxes(X_valid, 1, 3)

print X_valid.shape
print (Y).shape

Xtrain = X[0:2000,:,:,:]
Ytrain = Y[0:2000,:]

print Xtrain.shape
print Ytrain.shape

(2000, 3, 224, 224)
(23000, 2)
(2000, 3, 224, 224)
(2000, 2)


In [ ]:
validation = (X_valid, Y_valid)

print len(Xtrain)
print len(Ytrain)
model.compile(
        # loss='binary_crossentropy',
       #optimizer=SGD(lr=10.0),
          #optimizer=RMSprop(),
       optimizer=Adam(lr=0.0001),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
model.fit(Xtrain,Ytrain, epochs=1, batch_size=50, validation_data = validation) 

In [ ]:
def generate_train_arrays():
    for i in range(len(X)):
        yield (X[50*i:50*i+50,:,:,:],Y[50*i:50*i+50,:])

def generate_valid_arrays():
    for i in range(len(X_valid)):
        yield (X_valid[50*i:50*i+50,:,:,:],Y_valid[50*i:50*i+50,:]) 

model.load_weights("model_generator.h5")
model.fit_generator(generate_train_arrays(), steps_per_epoch=40, epochs=1, validation_steps = 40, validation_data = generate_valid_arrays())
model.save_weights("model_generator.h5")

In [ ]:

if os.path.exists('conv_valid_output.pckl') :
    conv_valid_output = restore('conv_valid_output.pckl')
else:
    conv_valid_output = convolutional_model.predict(X)
    store(conv_valid_output, 'conv_valid_output.pckl')
print (conv_valid_output.shape)

In [ ]:
def generate_train_arrays():
    for i in range(len(X)):
        yield (X[50*i:50*i+50,:,:,:],Y[50*i:50*i+50,:])

def generate_valid_arrays():
    for i in range(len(X_valid)):
        yield (X_valid[50*i:50*i+50,:,:,:],Y_valid[50*i:50*i+50,:]) 

model.load_weights("model_generator.h5")
model.fit_generator(generate_train_arrays(), steps_per_epoch=40, epochs=1, validation_steps = 40, validation_data = generate_valid_arrays())
model.save_weights("model_generator.h5")